In [1]:
import os
import openai
import json
from tqdm import tqdm
import time
import re
import csv  
from typing import Any, List, Mapping, Optional
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoModel
from transformers.generation.utils import GenerationConfig
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
import requests
import json

[2023-11-08 12:30:17,538] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
openai.api_key = "sk-nllqSQZXUcE0KocTB28904481846487bA2FfF82a9039Ca03" #要更换成自已的API KEY
openai.api_base = "https://api.132999.xyz/v1"

prompt = "Please write down 20 specific items  \
from shopping malls, outputting their detailed  \
names, types, and one-sentence descriptions.When  \
the type belongs to multiple categories, separate \
different categories with |. Please output in the following format: \
Item: item name \n \
Type: item types (split with |) \n \
Description: item description in one sentence \n \
Item: item name \n \
Type: item types (split with |) \n \
Description: item description in one sentence \n \
..."

model_path = "/home/liuyijun27/pretrain_models/baichuan-inc/Baichuan2-13B-Chat-4bits"
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.bfloat16, trust_remote_code=True)
model.generation_config = GenerationConfig.from_pretrained(model_path)

def save_data(products, output_file="new_items.csv"):
    # 定义CSV文件的列头和数据  
    name_set = set()
    fields = ["id", "title", "genre", "description"]  
    rows = []  
    pattern = r"\d+\."  
    replacement = ""  
    
    # 生成CSV文件的数据行  
    for i, item in enumerate(products, start=0):  
        title = re.sub(pattern, replacement, item["title"])
        if(title in name_set):
            continue
        row = [i, title, item["genre"], item["description"]]  
        rows.append(row)  
        name_set.add(title)

    # 写入CSV文件  
    with open(output_file, mode="w", newline="") as file:  
        writer = csv.writer(file)  
        writer.writerow(fields)  # 写入列头  
        writer.writerows(rows)  # 写入数据行

def chat(prompt):
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content":prompt}
    ]
)
    answer = response.choices[0].message.content
    return answer

def chat_local(prompt):
    messages = []
    messages.append({"role": "user", "content": prompt})
    answer = model.chat(tokenizer, messages)
    return answer

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [3]:
products = []
for i in tqdm(range(50)):
    try:
        respone = chat(prompt)
        matche_names = re.findall(r'Item:(.*?)\nType:', respone)  
        matche_types = re.findall(r'Type:(.*?)\nDescription:', respone)  
        matche_descriptions = re.findall(r'Description:(.*?)\.', respone)
        assert len(matche_names) == len(matche_types) == len(matche_descriptions)
        for j in range(len(matche_names)):
            products.append({
                "title": matche_names[j],
                "genre": matche_types[j],
                "description": matche_descriptions[j],
            })
        if(i%1==0):
            save_data(products, "new_items.csv")
    except Exception as e:
        # 捕获其他类型的异常
        print("发生了错误:", e)
        time.sleep(5)


  2%|▏         | 1/50 [00:55<45:10, 55.32s/it]

[{'title': ' iPhone 11', 'genre': ' Electronics', 'description': ' The latest smartphone from Apple with a powerful processor and improved cameras'}, {'title': ' Adidas Superstar sneakers', 'genre': ' Footwear | Fashion', 'description': ' Classic and iconic sneakers with a distinctive shell toe design, perfect for sports and casual wear'}, {'title': " Levi's 501 jeans", 'genre': ' Clothing | Fashion', 'description': ' Timeless and versatile jeans that are known for their high quality and classic straight leg fit'}, {'title': ' Sony PlayStation 5', 'genre': ' Electronics | Gaming', 'description': ' The next-generation gaming console with groundbreaking graphics and immersive gaming experiences'}, {'title': ' Michael Kors handbag', 'genre': ' Accessories | Fashion', 'description': ' Stylish and luxurious handbag designed by Michael Kors, perfect for adding a touch of elegance to any outfit'}, {'title': ' Nikon D850 DSLR camera', 'genre': ' Electronics | Photography', 'description': ' Hig

  4%|▍         | 2/50 [01:51<44:32, 55.67s/it]

[{'title': ' iPhone 11', 'genre': ' Electronics', 'description': ' The latest smartphone from Apple with a powerful processor and improved cameras'}, {'title': ' Adidas Superstar sneakers', 'genre': ' Footwear | Fashion', 'description': ' Classic and iconic sneakers with a distinctive shell toe design, perfect for sports and casual wear'}, {'title': " Levi's 501 jeans", 'genre': ' Clothing | Fashion', 'description': ' Timeless and versatile jeans that are known for their high quality and classic straight leg fit'}, {'title': ' Sony PlayStation 5', 'genre': ' Electronics | Gaming', 'description': ' The next-generation gaming console with groundbreaking graphics and immersive gaming experiences'}, {'title': ' Michael Kors handbag', 'genre': ' Accessories | Fashion', 'description': ' Stylish and luxurious handbag designed by Michael Kors, perfect for adding a touch of elegance to any outfit'}, {'title': ' Nikon D850 DSLR camera', 'genre': ' Electronics | Photography', 'description': ' Hig

  8%|▊         | 4/50 [03:55<45:51, 59.81s/it]

...

 96%|█████████▌| 48/50 [44:02<01:56, 58.29s/it]

发生了错误: HTTP code 504 from API (<html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx</center>
</body>
</html>
)


 98%|█████████▊| 49/50 [45:07<01:00, 60.36s/it]

发生了错误: HTTP code 504 from API (<html>
<head><title>504 Gateway Time-out</title></head>
<body>
<center><h1>504 Gateway Time-out</h1></center>
<hr><center>nginx</center>
</body>
</html>
)


100%|██████████| 50/50 [46:12<00:00, 55.45s/it]


In [4]:
import re  
  
# 假设这是你的文本  
text = """  
Item: Yoga mat
Type: Sports | Health
Description: A cushioned mat used during yoga and other exercises to provide comfort and support for the body.

Item: Coffee maker
Type: Appliances | Kitchen
Description: A device used for brewing coffee by passing hot water through ground coffee beans to extract flavor.
"""  
  
# 定义一个正则表达式匹配Item:和Type:之间的内容  
matches = re.findall(r'Type:(.*?)\nDescription:', text)  
matches = re.findall(r'Description:(.*?)\n\nItem:', text)  
  
# 输出匹配结果  
for match in matches:  
    print(match)

 A cushioned mat used during yoga and other exercises to provide comfort and support for the body.


In [5]:
save_data(products, "new_items.csv")

In [1]:
import pandas as pd  
import re
# 读取CSV文件  
df = pd.read_csv('item-v1.csv')  
  
# 添加新的列'item_detail'，其值为'title'列的值加上"'s detail"  
df['title'] = df['title'].apply(lambda x: re.sub(r'^\s*\d+\.\s*', '', x))  
df['item_detail'] = df['title'].apply(lambda x: x + "'s detail")  

  
# 保存到新的CSV文件  
df.to_csv('item.csv', index=False)

In [ ]:
import pandas as pd  
import re
# 读取CSV文件  
df = pd.read_csv('item.csv')  
for index, row in tqdm(df.iterrows(),total=df.shape[0]):  
    if(row['item_detail']==row['title']+ "'s detail"):
        item_name = row['title']
        prompt = f"Given the product name: {item_name}, please provide detailed information about this product, including various product parameters and the specific price information, directly output the detailed information content (no more than 50 words): "
        try:
            print("\nPrompt: ",prompt)
            respones = chat_local(prompt)
            print("Out: ",respones)
            df.iat[index, df.columns.get_loc('item_detail')] = respones.replace("\n","")
        except Exception as e:
            # 捕获其他类型的异常
            print("发生了错误:", e)
            time.sleep(5)

df.to_csv('item.csv', index=False)

  0%|          | 0/660 [00:00<?, ?it/s]


Prompt:  Given the product name:  iPhone 11, please provide detailed information about this product, including various product parameters and the specific price information, directly output the detailed information content (no more than 50 words): 


  0%|          | 1/660 [00:07<1:25:28,  7.78s/it]

Out:  The iPhone 11 is a smartphone developed by Apple with a 6.1-inch LCD screen, a triple camera system, and a powerful A13 Bionic chip. It supports dual-mode 4G networks and has a battery life of up to 17 hours. The starting price is $699.

Prompt:  Given the product name:  Adidas Superstar sneakers, please provide detailed information about this product, including various product parameters and the specific price information, directly output the detailed information content (no more than 50 words): 


  0%|          | 2/660 [00:14<1:18:54,  7.20s/it]

...

Out:  Product Name: Formal Suit
Detailed Information: This is a high-quality formal suit designed for professional and business occasions. It consists of a jacket and pants made of fine fabric with elegant design. The jacket features a notched lapel, two front pockets, and two buttoned cuffs. Pants have two slanted pockets and a buttoned waist. Available in different sizes and colors to meet your needs. Price: $299.

Prompt:  Given the product name:  T-shirt, please provide detailed information about this product, including various product parameters and the specific price information, directly output the detailed information content (no more than 50 words): 


 28%|██▊       | 183/660 [22:25<1:05:05,  8.19s/it]

Out:  Product Name: T-shirt
Detailed Information: The T-shirt is a popular clothing item, suitable for both men and women. It is usually made of cotton or polyester materials with different colors and patterns available. This product comes in different sizes, styles, and designs to suit personal preferences. Prices vary depending on the quality, brand, and material used.

Prompt:  Given the product name:  Running shoes, please provide detailed information about this product, including various product parameters and the specific price information, directly output the detailed information content (no more than 50 words): 


 28%|██▊       | 184/660 [22:39<1:18:16,  9.87s/it]

Out:  Product Name: Running Shoes
- Material: High quality rubber sole and breathable mesh upper.
- Weight: Approx. 300g per pair.
- Heel height: 4cm/1.5 inches.
- Insole: Removable, antimicrobial, shock-absorbing insole.
- Outsole: Rigid, durable, anti-slip pattern.
- Upper: Lightweight, breathable, supportive mesh fabric.
- Sizes: Available in US men's sizes 6 to 12.
Price: $50 per pair.

Prompt:  Given the product name:  Smartphone, please provide detailed information about this product, including various product parameters and the specific price information, directly output the detailed information content (no more than 50 words): 
